In [34]:
import pandas as pd
from collections import Counter

#from stat_rating_model import m_opp_stat_ratings

In [35]:
m_compact_results = pd.read_csv('data/men/MRegularSeasonCompactResults.csv')
w_compact_results = pd.read_csv('data/women/WRegularSeasonCompactResults.csv')
m_team_conferences = pd.read_csv('data/men/MTeamConferences.csv')
w_team_conferences = pd.read_csv('data/women/WTeamConferences.csv')
m_team_conferences['ConfAbbrev'] = 'm_' + m_team_conferences['ConfAbbrev']
w_team_conferences['ConfAbbrev'] = 'w_' + w_team_conferences['ConfAbbrev']


compact_results = pd.concat([m_compact_results, w_compact_results], ignore_index=True)
team_conferences = pd.concat([m_team_conferences, w_team_conferences], ignore_index=True)

compact_results = compact_results.merge(team_conferences, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
compact_results.rename(columns={'ConfAbbrev': 'WConfAbbrev'}, inplace=True)
compact_results.drop('TeamID', axis=1, inplace=True)
compact_results = compact_results.merge(team_conferences, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
compact_results.rename(columns={'ConfAbbrev': 'LConfAbbrev'}, inplace=True)
compact_results.drop('TeamID', axis=1, inplace=True)
compact_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WConfAbbrev,LConfAbbrev
0,1985,20,1228,81,1328,64,N,0,m_big_ten,m_big_eight
1,1985,25,1106,77,1354,70,H,0,m_swac,m_meac
2,1985,25,1112,63,1223,56,H,0,m_pac_ten,m_a_sun
3,1985,25,1165,70,1432,54,H,0,m_ivy,m_ind
4,1985,25,1192,86,1447,74,H,0,m_ecacm,m_ecacm
...,...,...,...,...,...,...,...,...,...,...
327739,2025,106,3242,63,3416,58,H,0,w_big_twelve,w_big_twelve
327740,2025,106,3329,68,3428,64,A,0,w_big_twelve,w_big_twelve
327741,2025,106,3349,72,3194,39,H,0,w_aac,w_aac
327742,2025,106,3378,70,3150,52,A,0,w_aac,w_aac


In [36]:
w_team_conferences

,Season,TeamID,ConfAbbrev
0,1998,3102,w_wac
1,1998,3103,w_mac
2,1998,3104,w_sec
3,1998,3106,w_swac
4,1998,3108,w_swac
...,...,...,...
9485,2025,3476,w_nec
9486,2025,3477,w_southland
9487,2025,3478,w_nec
9488,2025,3479,w_nec


In [37]:
conf_rows = []
for y in team_conferences['Season'].unique():
    year_results = compact_results[compact_results['Season'] == y].copy()
    conferences = set(year_results['WConfAbbrev'].tolist() + year_results['LConfAbbrev'].tolist())

    inter_conference_year_results = year_results[year_results['WConfAbbrev'] != year_results['LConfAbbrev']]

    # Build Conf Win Percent Dict
    conf_win_percentages = {}
    for conf in conferences:
        inter_conf_wins = len(inter_conference_year_results[inter_conference_year_results['WConfAbbrev'] == conf])
        inter_conf_losses = len(inter_conference_year_results[inter_conference_year_results['LConfAbbrev'] == conf])
        inter_win_percentage = inter_conf_wins / (inter_conf_wins + inter_conf_losses) if (inter_conf_wins + inter_conf_losses) > 0 else None

        conf_win_percentages[conf] = inter_win_percentage


    for conf in conferences:
        # Inter-conference record
        inter_conf_wins = len(inter_conference_year_results[inter_conference_year_results['WConfAbbrev'] == conf])
        inter_conf_losses = len(inter_conference_year_results[inter_conference_year_results['LConfAbbrev'] == conf])
        inter_win_percentage = inter_conf_wins / (inter_conf_wins + inter_conf_losses) if (inter_conf_wins + inter_conf_losses) > 0 else None

        # Count appearances of each team that played against this conference
        opponent_counts = Counter(inter_conference_year_results[inter_conference_year_results['LConfAbbrev'] == conf]['WTeamID'].tolist() + inter_conference_year_results[inter_conference_year_results['WConfAbbrev'] == conf]['LTeamID'].tolist())

        # Compute weighted total wins and losses by those opponents
        inter_wins_by_opponents = sum(
            opponent_counts[team] * len(inter_conference_year_results[inter_conference_year_results['WTeamID'] == team])
            for team in opponent_counts
        )
        inter_losses_by_opponents = sum(
            opponent_counts[team] * len(inter_conference_year_results[inter_conference_year_results['LTeamID'] == team])
            for team in opponent_counts
        )

        opp_inter_win_percentage = inter_wins_by_opponents / (inter_wins_by_opponents + inter_losses_by_opponents) if (inter_wins_by_opponents + inter_losses_by_opponents) > 0 else None

        # Count appearances of each conference that played against this conference
        opponent_conf_counts = Counter(inter_conference_year_results[inter_conference_year_results['LConfAbbrev'] == conf]['WConfAbbrev'].tolist() + inter_conference_year_results[inter_conference_year_results['WConfAbbrev'] == conf]['LConfAbbrev'].tolist())

        total_games = sum(opponent_counts.values())
        weighted_opp_conf_win_percentage = sum(
            (opponent_conf_counts[opp_conf] / total_games) * conf_win_percentages[opp_conf]
            for opp_conf in opponent_conf_counts if conf_win_percentages[opp_conf] is not None
        ) if total_games > 0 else None

        conf_rows.append([y, conf, inter_conf_wins, inter_conf_losses, inter_win_percentage, inter_wins_by_opponents, inter_losses_by_opponents, opp_inter_win_percentage, weighted_opp_conf_win_percentage])

inter_conference_play = pd.DataFrame(conf_rows, columns=['Season', 'ConfAbbrev', 'Wins', 'Losses', 'WinPercentage', 'OpponentWins', 'OpponentLosses', 'OpponentWinPercentage', 'WeightedOpponentConfWinPercentage'])

inter_conference_play

,Season,ConfAbbrev,Wins,Losses,WinPercentage,OpponentWins,OpponentLosses,OpponentWinPercentage,WeightedOpponentConfWinPercentage
0,1985,m_maac,50,47,0.515464,534,593,0.473824,0.448740
1,1985,m_metro,52,37,0.584270,585,475,0.551887,0.561291
2,1985,m_ecacs,42,43,0.494118,448,552,0.448000,0.468052
3,1985,m_gsc,24,45,0.347826,398,434,0.478365,0.458517
4,1985,m_mvc,48,32,0.600000,525,427,0.551471,0.501728
...,...,...,...,...,...,...,...,...,...
2223,2025,w_cusa,49,47,0.510417,537,517,0.509488,0.478334
2224,2025,w_patriot,56,50,0.528302,504,662,0.432247,0.438378
2225,2025,w_big_sky,43,53,0.447917,518,494,0.511858,0.491456
2226,2025,w_mac,61,66,0.480315,661,653,0.503044,0.497608


In [38]:
m_tourney_seeds = pd.read_csv('data/men/ncaa_tournament/MNCAATourneySeeds.csv')
w_tourney_seeds = pd.read_csv('data/women/ncaa_tournament/WNCAATourneySeeds.csv')
tourney_seeds = pd.concat([m_tourney_seeds, w_tourney_seeds], ignore_index=True)

tourney_seeds = tourney_seeds.merge(team_conferences, how='left', on=['Season', 'TeamID'])
lookback_years = 5
conf_bids = tourney_seeds.groupby(['Season', 'ConfAbbrev']).size().reset_index(name='NCAATournamentBids')
for i in range(1, lookback_years + 1):
    conf_bids[f'LookbackBids{i}'] = conf_bids.groupby('ConfAbbrev')['NCAATournamentBids'].shift(i)
    # TODO Has nans where conference didn't exist? Maybe turn these to zeros to affect average?
conf_bids['AvgRecentBids'] = conf_bids[[f'LookbackBids{i}' for i in range(1, lookback_years + 1)]].mean(axis=1)
conf_bids

,Season,ConfAbbrev,NCAATournamentBids,LookbackBids1,LookbackBids2,LookbackBids3,LookbackBids4,LookbackBids5,AvgRecentBids
0,1985,m_a_sun,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,m_a_ten,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1985,m_acc,5,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,m_big_east,6,NaN,NaN,NaN,NaN,NaN,NaN
4,1985,m_big_eight,3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022,2024,w_summit,1,1.0,1.0,2.0,2.0,1.0,1.4
2023,2024,w_sun_belt,1,1.0,1.0,1.0,1.0,1.0,1.0
2024,2024,w_swac,1,1.0,1.0,1.0,1.0,1.0,1.0
2025,2024,w_wac,1,1.0,1.0,1.0,1.0,1.0,1.0


In [39]:
conf_features = inter_conference_play[['Season', 'ConfAbbrev', 'WinPercentage', 'OpponentWinPercentage', 'WeightedOpponentConfWinPercentage']].merge(conf_bids[['Season', 'ConfAbbrev', 'NCAATournamentBids', 'LookbackBids1', 'AvgRecentBids']], how='left', on=['Season', 'ConfAbbrev'])
conf_features

,Season,ConfAbbrev,WinPercentage,OpponentWinPercentage,WeightedOpponentConfWinPercentage,NCAATournamentBids,LookbackBids1,AvgRecentBids
0,1985,m_maac,0.515464,0.473824,0.448740,1.0,NaN,NaN
1,1985,m_metro,0.584270,0.551887,0.561291,2.0,NaN,NaN
2,1985,m_ecacs,0.494118,0.448000,0.468052,1.0,NaN,NaN
3,1985,m_gsc,0.347826,0.478365,0.458517,NaN,NaN,NaN
4,1985,m_mvc,0.600000,0.551471,0.501728,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2223,2025,w_cusa,0.510417,0.509488,0.478334,NaN,NaN,NaN
2224,2025,w_patriot,0.528302,0.432247,0.438378,NaN,NaN,NaN
2225,2025,w_big_sky,0.447917,0.511858,0.491456,NaN,NaN,NaN
2226,2025,w_mac,0.480315,0.503044,0.497608,NaN,NaN,NaN


In [40]:
team_conferences.to_csv('my_combined_conferences.csv', index=False)
conf_features.to_csv('conference_features.csv', index=False)